<a href="https://colab.research.google.com/github/denix56/Keras-GAN/blob/master/GANmera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-06-17 08:11:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.231.75.48, 52.2.175.150, 34.206.36.121, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.231.75.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  7.20MB/s    in 2.2s    

2019-06-17 08:11:49 (7.20 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
!git clone https://github.com/keras-team/keras-contrib.git
sys.path.append('/content/keras-contrib')

Cloning into 'keras-contrib'...
remote: Enumerating objects: 3622, done.
remote: Total 3622 (delta 0), reused 0 (delta 0), pack-reused 3622
Receiving objects: 100% (3622/3622), 858.42 KiB | 1.16 MiB/s, done.
Resolving deltas: 100% (2322/2322), done.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.mkdir('logs')

NameError: ignored

In [0]:
LOG_DIR = '/content/logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host localhost --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ea7d636a.ngrok.io


In [0]:
!unzip "/content/gdrive/My Drive/PhotoQualityDataset.zip" -d /content/datasets/

Archive:  /content/gdrive/My Drive/PhotoQualityDataset.zip
   creating: /content/datasets/PhotoQualityDataset/
   creating: /content/datasets/PhotoQualityDataset/HighQuality/
   creating: /content/datasets/PhotoQualityDataset/HighQuality/animal/
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/160944.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161031.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161065.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161072.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161116.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161156.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161175.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161178.jpg  
  inflating: /content/datasets/PhotoQualityDataset/HighQuality/animal/161205.jpg  
  infla

In [0]:
import shutil
os.chdir('/content/datasets/PhotoQualityDataset/')
os.mkdir('trainA')
os.mkdir('trainB')

shutil.copytree('LowQuality/architecture', 'trainA/architecture')
shutil.copytree('LowQuality/landscape', 'trainA/landscape')

shutil.copytree('HighQuality/architecture', 'trainB/architecture')
shutil.copytree('HighQuality/landscape', 'trainB/landscape')

In [0]:
import scipy
from pathlib import Path
import numpy as np
from random import shuffle
from math import floor
import os
import shutil
import cv2

def get_training_and_testing_sets(file_list):
    split = 0.7
    split_index = floor(len(file_list) * split)
    training = file_list[:split_index]
    testing = file_list[split_index:]
    return training, testing

def randomize_files(file_list):
    shuffle(file_list)

class DataLoader():
    def __init__(self, dataset_path, img_res=(128, 128)):
        self.dataset_path = dataset_path
        self.img_res = img_res

    def __create_train_test(self, domain):
        file_list = list(Path(self.dataset_path).glob('train{}/**/*.jpg'.format(domain)))
        randomize_files(file_list)
        _, file_list_test = get_training_and_testing_sets(file_list)

        for filename in file_list_test:
            dest_filename = str(filename).replace('train', 'test')
            os.makedirs(os.path.dirname(dest_filename), exist_ok=True)
            shutil.move(filename, dest_filename)

    def create_train_test(self):
        self.__create_train_test('A')
        self.__create_train_test('B')

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = list(Path(self.dataset_path).glob('{}/**/*.jpg'.format(data_type)))
        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = self.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = self.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = list(Path(self.dataset_path).glob('{}A/**/*.jpg'.format(data_type)))
        path_B = list(Path(self.dataset_path).glob('{}B/**/*.jpg'.format(data_type)))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = self.imresize(img_A, self.img_res)
                img_B = self.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = self.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
      return cv2.imread(str(path)).astype(np.float32)
    
    def imresize(self, data, res):
      return cv2.resize(data, res)
      


In [0]:
data_loader = DataLoader(dataset_path='/content/datasets/PhotoQualityDataset')
data_loader.create_train_test()

In [0]:
os.chdir('/content')

In [0]:
from __future__ import print_function, division
import scipy
import scipy.misc

from keras.datasets import mnist
import keras.backend as K
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import LearningRateScheduler, History, TensorBoard, ModelCheckpoint, Callback
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Extending the ZeroPadding2D layer to do reflection padding instead.
class ReflectionPadding2D(ZeroPadding2D):
    def call(self, x, mask=None):
        pattern = [[0, 0],
                   self.padding[0],
                   self.padding[1],
                   [0, 0]]
        return tf.pad(x, pattern, mode='REFLECT')

class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'PhotoQualityDataset'
        self.data_loader = DataLoader(dataset_path='/content/datasets/PhotoQualityDataset',
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        self.d_A.summary()

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        self.g_AB.summary()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    root_mean_squared_error, root_mean_squared_error,
                                    root_mean_squared_error, root_mean_squared_error],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=3, stride=2, relu=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=stride, padding='same')(layer_input)
            if relu:
              d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        def residual(layer_input, filters, f_size=3, stride=1):
            d = conv2d(layer_input, filters, f_size=f_size, stride=stride)
            concat = Concatenate()([d, layer_input])
            d = conv2d(concat, filters, f_size=f_size, stride=stride)
            concat = Concatenate()([d, concat])
            d = conv2d(concat, filters, f_size=f_size, stride=stride)
            concat = Concatenate()([d, concat])
            d = conv2d(concat, filters, f_size=1, stride=1, relu=False)
            d = Concatenate()([d, layer_input])
            return d

        def resize_conv2d(layer_input, filters, f_size=3, stride=1):
            u = UpSampling2D(size=2, interpolation='nearest')(layer_input)
            u = ReflectionPadding2D()(u)
            u = Conv2D(filters, kernel_size=f_size, strides=stride, padding='valid')(u)
            u = LeakyReLU(alpha=0.2)(u)
            u = InstanceNormalization()(u)
            return u


        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, f_size=7, stride=1)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)

        residual_filters = self.gf*4

        # Residual learning
        r1 = residual(d3, residual_filters)
        r2 = residual(r1, residual_filters)
        r3 = residual(r2, residual_filters)
        #r4 = residual(r3, residual_filters)
        #r5 = residual(r4, residual_filters)
        #r6 = residual(r5, residual_filters)

        # Upsampling
        u1 = resize_conv2d(r3, self.gf*2)
        u2 = resize_conv2d(u1, self.gf)

        output_img = Conv2D(self.channels,
                            kernel_size=7,
                            strides=1,
                            padding='same',
                            activation='tanh',
                            kernel_regularizer=regularizers.l1())(u2)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, stride=2, normalization=True, relu=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=stride, padding='same')(layer_input)
            if relu:
                d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = d_layer(d4, 1, 4, stride=1, relu=False)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        tb_callback = TensorBoard(log_dir='./logs/combined', batch_size=batch_size, write_grads=False, write_images=True, write_graph=True)
        tb_callback.set_model(self.combined)
        
        tb_callback_genA = TensorBoard(log_dir='./logs/generatorAB', batch_size=batch_size, write_grads=False, write_images=False, write_graph=True)
        tb_callback_genA.set_model(self.g_AB)
        
        tb_callback_genB = TensorBoard(log_dir='./logs/generatorBA', batch_size=batch_size, write_grads=False, write_images=False, write_graph=True)
        tb_callback_genB.set_model(self.g_BA)

        tb_callback_discA = TensorBoard(log_dir='./logs/discriminatorA', batch_size=batch_size, write_grads=False, write_images=False, write_graph=True)
        tb_callback_discA.set_model(self.d_A)
        
        tb_callback_discB = TensorBoard(log_dir='./logs/discriminatorB', batch_size=batch_size, write_grads=False, write_images=False, write_graph=True)
        tb_callback_discB.set_model(self.d_B)

        #tbi_callback = TensorBoardImage(self.generator, self.data_loader, batch_size, log_dir='./images/PhotoQualityDataset')        

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i, self.data_loader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                    #tbi_callback.on_epoch_end(epoch)
                    
                #loss_dict = dict(zip(['Total', 'mse1', 'mse2',
                 #                                    'rmse1', 'rmse2','rmse3','rmse4'], g_loss))
            #checkpoint_cb.on_epoch_end(epoch)
                #tb_callback.on_epoch_end(epoch,loss_dict)
                    

        tb_callback.on_train_end()
        tbi_callback.on_train_end()

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()


gan = CycleGAN()
gan.train(epochs=200, batch_size=1, sample_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_73 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_553 (Conv2D)          (None, 64, 64, 64)        3136      
_________________________________________________________________
leaky_re_lu_499 (LeakyReLU)  (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_554 (Conv2D)          (None, 32, 32, 128)       131200    
_________________________________________________________________
leaky_re_lu_500 (LeakyReLU)  (None, 32, 32, 128)       0         
_________________________________________________________________
instance_normalization_505 ( (None, 32, 32, 128)       2         
_________________________________________________________________
conv2d_555 (Conv2D)          (None, 16, 16, 256)       524544    
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/200] [Batch 0/990] [D loss: 1.498318, acc:  30%] [G loss: 28.387894, adv: 1.997467, recon: 0.941295, id: 0.650774] time: 0:03:03.148779 


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/200] [Batch 1/990] [D loss: 1.498152, acc:  26%] [G loss: 27.566179, adv: 1.995649, recon: 0.851921, id: 1.348952] time: 0:03:04.584579 
[Epoch 0/200] [Batch 2/990] [D loss: 1.496315, acc:  27%] [G loss: 27.078938, adv: 1.994365, recon: 0.846933, id: 1.091392] time: 0:03:05.666999 
[Epoch 0/200] [Batch 3/990] [D loss: 1.494656, acc:  33%] [G loss: 26.156292, adv: 1.991704, recon: 0.786805, id: 1.303263] time: 0:03:06.759035 
[Epoch 0/200] [Batch 4/990] [D loss: 1.492893, acc:  30%] [G loss: 29.354263, adv: 1.988990, recon: 0.949029, id: 1.087196] time: 0:03:07.840642 
[Epoch 0/200] [Batch 5/990] [D loss: 1.491305, acc:  26%] [G loss: 29.873972, adv: 1.986438, recon: 0.985155, id: 1.064608] time: 0:03:08.932660 
[Epoch 0/200] [Batch 6/990] [D loss: 1.489746, acc:  29%] [G loss: 30.072968, adv: 1.984613, recon: 0.988386, id: 1.121465] time: 0:03:10.032490 
[Epoch 0/200] [Batch 7/990] [D loss: 1.486726, acc:  30%] [G loss: 27.714537, adv: 1.983883, recon: 0.874641, id: 1.064814] 